In [ ]:
from IPython.display import display, HTML; display(HTML("<style>.container { width:90% !important; }</style>")) 

# Web scraping i JSON API

Do scrapingu danych z internetu korzystamy z dwóch bibliotek, które pozwalają na:

1. Wysyłanie zapytań HTTP w programie Pythona (zamiast w przeglądarce) - `requests`
2. Interpretację odpowiedzi HTTP tak aby ułatwić jej przeszukiwanie - `bs4` (`BeautifulSoup`)

In [ ]:
import requests
from bs4 import BeautifulSoup

## Środowisko do nauki scrapingu

https://toscrape.com/

oraz podstrony:

https://quotes.toscrape.com/

https://books.toscrape.com/

## Podstawowe komendy

### Pobieranie kodu HTML

In [ ]:
quotes_url = "https://quotes.toscrape.com/"

response = requests.get(quotes_url)
response

In [ ]:
print(response.text)

In [ ]:
type(response.text)

In [ ]:
soup = BeautifulSoup(response.text, 'html.parser')
soup

In [ ]:
type(soup)

### Parsowanie kodu HTML

#### Ekstrakcja tekstu wewnątrz znaczników HTML

In [ ]:
soup.find("a")

In [ ]:
soup.find_all("a")

In [ ]:
len(soup.find_all("a"))

In [ ]:
first_anchor_tag = soup.find("a")
first_anchor_tag

In [ ]:
page_header = first_anchor_tag.text
page_header

---

In [ ]:
soup.find("h1") #.find("a")

#### Doprecyzowanie znacznika poprzez podanie klasy

In [ ]:
soup.find_all("a", class_="tag")   # oprócz klasy można podawać też inne atrybuty, np. id

#### Wyciąganie wartości atrybutów

In [ ]:
soup.find_all("a")[3]

In [ ]:
soup.find_all("a")[3].attrs

In [ ]:
soup.find_all("a")[3].attrs["href"]

In [ ]:
soup.find_all("a")[3].attrs["class"]

In [ ]:
soup.find_all("a")[3]["class"]

#### Szukanie elementów na podstawie dowolnych atrybutów

In [ ]:
soup.find_all("div", {'itemtype': 'http://schema.org/CreativeWork'})

In [ ]:
soup.find_all("div", {'class': 'quote'})

#### Narzędzia developerskie - devtools

Aby poznać strukturę kodu HTML jeszcze przed scrapingiem wykorzystajmy narządzia developerskie (devtoolsy) 

### Podsumowanie podstawowych poleceń poleceń
#### Pobieranie i parsowanie HTML

In [ ]:
url = "https://example.org"

response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

#### Przeszukiwanie kodu HTML

In [ ]:
soup.find("div")                     # znajdź pierwszy znacznik 'div' jaki napotkasz
soup.find_all("a")                   # znajdź wszystkie znaczniki 'a'
soup.find_all("a", "class_name")     # znajdź wszystkie znaczniki 'a' z klasą class_name
soup.find_all("p")[3].text           # spośród wszystkich znaczników 'p' weź ten o indeksie 3 i wyciągnij jego zawartość
soup.find("span").attrs["class"]     # znajdź pierwszy znacznik 'span' i wyciągnij nazwę jego klasy
soup.find("h1").find("a")            # znajdź znacznik "a" wewnątrz znacznika "h1"

### Czas na samodzielne przyswojenie wiedzy
Zanim przejdziemy dalej poświęćmy 10 minut na samodzielne eksperymenty oraz ewentualne pytania

## Pobieranie cytatów - zadania rozgrzewkowe
### Treść pojedynczego cytatu
Wyciągnijmy treść pojedynczego cytatu.

Aby wiedzieć czego dokładnie szukamy w kodzie HTML użyjemy devtoolsów przeglądarki.

In [ ]:
quotes_url = "https://quotes.toscrape.com/"
response = requests.get(quotes_url)
soup = BeautifulSoup(response.text, 'html.parser')

In [ ]:
# Wyciągnijmy pierwszy (zerowy) cytat i zapiszmy go do zmiennej
quote = soup.find_all("div", class_="quote")[0]
quote

In [ ]:
# Wychodząc od tego cytatu szukajmy dalej - treść cytatu znajduje się w znaczniku 'span'
text = quote.find("span", class_="text").text
text

In [ ]:
# możemy również zrobić to krócej - jednak nie zawsze bezpośrednie odwołanie zadziała i jeśli element jest bardzo zagnieżdżony należy wyszukiwać go po kolei

soup.find("span", "text").text   

### Lista list tagów

Stwórzmy listę, której elementy będą odpowiadały kolejnym cytatom. Każdy element to lista tagów w danym cytacie

In [ ]:
nested_list_of_tags = []                               # chcemy żeby wyglądało to tak: [[tag1, tag2, tag3], [tag2, tag4], ..., [tag5, tag1, tag6]]

for quote in soup.find_all("div", class_="quote"):     # pętla po wszystkich cytatach
    nested_list_of_tags.append([])                     # kazdy kolejny cytat to nowa wewnętrzna lista (lista tagów)
    
    for tag in quote.find_all("a", class_="tag"):      # pętla po wszystkich tagach w obrębie cytatu
        nested_list_of_tags[-1].append(tag.text)       # każdy tag danego cytatu dodajemy do wewnętrznej listy

In [ ]:
nested_list_of_tags

### Lista list tagów z kilku podstron

Cytaty znajdują się na kolejnych podstronach. Przejdźmy do następnej strony i pobierzmy kolejne. 

In [ ]:
nested_list_of_tags = []

for i in range(5):                                         # pętla po kilku podstronach
    for quote in soup.find_all("div", class_="quote"):
        nested_list_of_tags.append([])
    
        for tag in quote.find_all("a", class_="tag"):
            nested_list_of_tags[-1].append(tag.text)
        
    # Zebraliśmy już wszystkie informacje na tej podstronie. Przejdźmy teraz do następnej
    sub_url = soup.find("li", class_="next").find("a").attrs["href"]
    next_page_url = quotes_url + sub_url
    response = requests.get(next_page_url)
    soup = BeautifulSoup(response.text, 'html.parser')

In [ ]:
nested_list_of_tags

### Więcej informacji o cytatach - tekst, autor oraz lista tagów

Utwórzmy strukturę danych, w której przechowamy zarówno tekst jak i autora oraz tagi poszczególnych cytatów.

`[{"text": "text of the quote", "author": "Quote Author", "tags": ["tag1", "tag2", "tag3"]},
 ...
 ]`

In [ ]:
quotes_url = "https://quotes.toscrape.com/"
response = requests.get(quotes_url)
soup = BeautifulSoup(response.text, 'html.parser')

quotes_list = []
for i in range(5):
    for quote in soup.find_all("div", class_="quote"):
        
        single_quote = dict()
        
        single_quote["text"] = quote.find("span", class_="text").text.replace('“', '').replace('”', '')
        
        single_quote["author"] = quote.find("small", class_="author").text
        
        tags = quote.find_all("a", "tag")
        tags_texts = [t.text for t in tags]
        single_quote["tags"] = tags_texts
        
        quotes_list.append(single_quote)
        
    sub_url = soup.find("li", class_="next").find("a").attrs["href"]
    next_page_url = quotes_url + sub_url
    response = requests.get(next_page_url)
    soup = BeautifulSoup(response.text, 'html.parser')

In [ ]:
quotes_list

In [ ]:
quotes_list[0]

In [ ]:
quotes_list[14]

> **ZADANIA**

## JSON API

Czym jest API? 
- API to skrót od Application Programming Interface
- w zależności od konkretnego zastosowania może oznaczać trochę coś innego
- w naszym przypadku API jest pewną usługą webową, która pozwala pozyskać dane w ustrukturyzowanej postaci z bazy danych do której nie mamy bezpośredniego dostępu
- API to rodzaj pośrednika (interface) między bazą danych a naszą aplikacją

Jak działa API?
- możemy łączyć się z nim poprzez przeglądarkę lub wewnątrz naszego skryptu za pomocą biblioteki `requests`
- wysyłamy zapytanie typu `GET` a w odpowiedzi otrzymujemy dane w formacie JSON (czasem może to być również XML, ale tym nie będziemy się zajmować)

Czym jest JSON
- JavaScript Object Notation
- ustrukturyzowana forma zapisu danych
- JSON przypomina słownik w Pythonie. Dane przechowywane są na zasadzie klucz-wartość

### currencylayer API

**Przykładowe wywołanie API wygląda w następujacy sposób:**

http://api.currencylayer.com/historical?access_key=bef18344d09a9963fda9d0c8402ace0e&date=2020-12-12&currencies=EUR,PLN&format=1

Otwórzmy powyższy link żeby sprawdzić co się stanie.

Uwaga: 
1. Walutą odniesienia jest dolar amerykański. W darmowej wersji API ten parametr jest niemodyfikowalny
2. API zwykle mają limity requestów dla danego klucza. Szczegółowe informacje na ten temat powinny znajdować się w dokumentacji. W przypadku currencylayer jest to 250 wywołań na miesiąc (w planie darmowym)

**Struktura adresu URL**

Rozłóżmy URL na czynniki pierwsze:

http:// api.currencylayer.com/ historical?access_key= API_KEY &date= DATE &currencies= CURRENCIES_LIST &format=1

Nasze zapytanie precyzujemy podając wymagane argumenty. Informacje jak to zrobić możemy znaleźc w dokumentacji API. W tym przypadku potrzebujemy:
- klucz API
- datę
- listę walut jakie nas interesują

**Customizacja adresu URL:**

In [ ]:
API_KEY = # "bef18344d09a9963fda9d0c8402ace0e"  # wpisz swój klucz API
DATE = "2019-05-12"
CURRENCIES_LIST = "EUR,PLN"


my_url = f"http://api.currencylayer.com/historical?access_key={API_KEY}&date={DATE}&currencies={CURRENCIES_LIST}&format=1"
my_url

**Pobranie danych:**

In [ ]:
response = requests.get(my_url)
response_json = response.json()

response_json

In [ ]:
usd_per_pln = response_json["quotes"]["USDPLN"]
usd_per_eur = response_json["quotes"]["USDEUR"]

print(usd_per_pln)
print(usd_per_eur)

### OpenWeatherMap

https://openweathermap.org/

Przykładowe wywołanie:

http://api.openweathermap.org/data/2.5/forecast?q=Krakow&appid=7d0c48134ae346811fa50cf99109251f&units=metric

> **ZADANIA**